In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# # URL dari halaman arsip TurnBackHoax
# BASE_URL = "https://turnbackhoax.id/page"

# def scrape_turnbackhoax(start_page, end_page, output_file):
#     data = []

#     for page in range(start_page, end_page + 1):
#         print(f"Scraping halaman {page}...")
#         response = requests.get(BASE_URL + str(page))
#         if response.status_code != 200:
#             print(f"Halaman {page} tidak dapat diakses, melanjutkan ke halaman berikutnya.")
#             continue

#         soup = BeautifulSoup(response.text, 'html.parser')

#         # Ambil semua artikel di halaman
#         articles = soup.find_all('article', class_='mh-loop-item')
        
#         for article in articles:
#             try:
#                 # Judul dan URL
#                 title_elem = article.find('h3', class_='entry-title mh-loop-title')
#                 title = title_elem.text.strip() if title_elem else "No Title"
#                 url = title_elem.find('a')['href'] if title_elem and title_elem.find('a') else "No URL"

#                 # Excerpt (konten singkat dari halaman utama)
#                 excerpt_elem = article.find('div', class_='mh-excerpt')
#                 excerpt = excerpt_elem.text.strip() if excerpt_elem else "No Excerpt"

#                 # Mengakses halaman detail untuk mengambil konten lengkap dan tag
#                 if url != "No URL":
#                     detail_response = requests.get(url)
#                     detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

#                     # Konten lengkap
#                     fulltext_elem = detail_soup.find('div', class_='entry-content')
#                     fulltext = fulltext_elem.text.strip() if fulltext_elem else "No FullText"

#                     # Tags
#                     tags_section = detail_soup.find('span', class_='tags-links')
#                     tags = ", ".join([tag.text for tag in tags_section.find_all('a')]) if tags_section else "No Tags"
#                 else:
#                     fulltext = "No FullText"
#                     tags = "No Tags"

#                 # Tambahkan ke data
#                 data.append([title, excerpt, fulltext, tags, url])
#             except Exception as e:
#                 print(f"Error saat memproses artikel: {e}")

#     # Menyimpan data ke file CSV
#     with open(output_file, 'w', newline='', encoding='utf-8') as csv_file:
#         writer = csv.writer(csv_file)
#         writer.writerow(["Title", "Excerpt", "FullText", "Tags", "URL"])
#         writer.writerows(data)

#     print(f"Scraping selesai! Data disimpan di {output_file}")

# # Contoh penggunaan
# scrape_turnbackhoax(start_page=1, end_page=2, output_file='turnbackhoax_data.csv')


In [4]:
# Fungsi untuk scraping halaman detail berita
def scrape_detail_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Mengambil tanggal
    date_element = soup.find('span', class_='entry-meta-date updated')
    date = date_element.get_text(strip=True) if date_element else "Date not found"

    # Mengambil judul
    title_element = soup.find('h1', class_='entry-title')
    title = title_element.get_text(strip=True) if title_element else "Title not found"
    
    # Mengambil isi berita setelah garis pembatas
    content = []
    separator = soup.find('hr', class_='wp-block-separator has-alpha-channel-opacity')
    if separator:
        for sibling in separator.find_next_siblings():
            # Abaikan headline "Pemeriksaan Fakta"
            if sibling.name == 'h2' and "Pemeriksaan Fakta" in sibling.get_text(strip=True):
                continue
            
            # Tambahkan konten paragraf dan blockquote
            if sibling.name == 'blockquote':
                content.append(sibling.get_text(strip=True))
            elif sibling.name == 'p':
                content.append(sibling.get_text(strip=True))
            
            # Berhenti jika mencapai elemen dengan class kesimpulan atau paragraf terakhir
            if sibling.name == 'h2' and "Kesimpulan" in sibling.get_text(strip=True):
                break

    full_text = "\n".join(content).strip()
    
    return date, title, full_text

# URL contoh untuk scraping
urls = [
    'https://turnbackhoax.id/2024/12/04/salah-kisah-sukses-pemuda-pengangguran-berkat-main-judi-slot/',
]

# Menyimpan hasil ke file CSV
output_file = 'scraped_test.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['date', 'URL', 'Title', 'Full text']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for url in urls:
        try:
            date, title, full_text = scrape_detail_page(url)
            writer.writerow({'date': date, 'URL': url, 'Title': title, 'Full text': full_text})
            print(f"Scraped: {url}")
        except Exception as e:
            print(f"Error scraping {url}: {e}")

print(f"Data berhasil disimpan ke {output_file}")


Scraped: https://turnbackhoax.id/2024/12/04/salah-kisah-sukses-pemuda-pengangguran-berkat-main-judi-slot/
Data berhasil disimpan ke scraped_test.csv
